In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.exceptions import SpotifyException
import jwt


#this is where the user would need to enter their own credentials
sp_oauth = SpotifyOAuth(
    client_id = '73ab5349d4284b759c07843e5d202eff',
    client_secret = '6f02411e145b441dac805c627a2d90d4',
    redirect_uri='http://localhost:8888/callback',
    scope='playlist-read-private playlist-read-collaborative user-library-read'
)

sp = spotipy.Spotify(auth_manager=sp_oauth)

# so this is a function to list all of the user's playlists
#playlists that the user either created or followed
def list_user_playlists():
    playlist = sp.current_user_playlists()
    #can also put playlist in playlist["item"]
    for playlist in playlist["items"]:
        print(f"Playlist Name: {playlist['name']}, Playlist ID: {playlist['id']}")

list_user_playlists()


Playlist Name: Your Top Songs 2021, Playlist ID: 37i9dQZF1EUMDoJuT8yJsl
Playlist Name: Halloween Playlist 2024🎃, Playlist ID: 2bWtVAl9VX45x9gAUPiVOF
Playlist Name: Top 5000 Classic Rock Songs, Playlist ID: 2HMHJOy27G6N7K0jPkR7LP
Playlist Name: Top 1000 Classic Rock Songs, Playlist ID: 4V1V94JnfqltMWF612BVze
Playlist Name: Top 1000 Classic Rock Songs, Playlist ID: 3VEKmJXlvi8LERi7u2yhaH
Playlist Name: Club-Ready House Songs, Playlist ID: 5cSOHPSWTZkjjGVxsPEHTQ
Playlist Name: Club-Ready House Songs, Playlist ID: 5VPsVRxbt6H13s9MqISDyz
Playlist Name: Club-Ready House Songs, Playlist ID: 52WNi5AwO97mJfo6vSG6Is
Playlist Name: Club-Ready House Songs, Playlist ID: 6BIfwyCEOLXYn72cEKbpL0
Playlist Name: Club-Ready House Songs, Playlist ID: 0nmJ8HWAI1MwXnIEyABgn1
Playlist Name: Top 500 House Songs, Playlist ID: 0nY4xX58hLDQB4K9mOWgl0
Playlist Name: notice, Playlist ID: 3mzeVWd0uVg2VcSgA31VoE
Playlist Name: My Playlist #5, Playlist ID: 2NI96ZT0o8uMoaCJgScieA
Playlist Name: DJ, Playlist ID: 37i9dQ

In [ ]:
album = sp.current_user_saved_albums()
for album_item in album["items"]:
    print(f"album name {album_item['album']['name']}")

In [2]:
import pandas as pd
playlist_id = '2HfFccisPxQfprhgIHM7XH'

#get all the songs from the playlist_id
results = sp.playlist_tracks(playlist_id)
tracks = results['items']

#this while extract the info in results
#add to tracks
while results['next']:
    results = sp.next(results)
    tracks.extend(results['items'])


data = []
#get info like name, id, playlist, duration
for item in tracks:
    track = item['track']
    song_name = track['name']
    song_id = track['id']
    duration = track['duration_ms']
    artists = [artist['name'] for artist in track['artists']]
    release_date = sp.track(song_id)['album']['release_date']

    
    artist_id = track['artists'][0]['id']
    artist_info = sp.artist(artist_id)
    genres = artist_info['genres']

    # Append the data
    data.append({
        'Song Name': song_name,
        'Song ID': song_id,
        'Artists': artists,
        'Release Date': release_date,
        'Genres': genres,
        'Time (ms)':duration // 1000
    })


df = pd.DataFrame(data)


print(df)
df.to_csv('playlist_songs.csv', index=False)

                                            Song Name                 Song ID  \
0                                          Wonderwall  1qPbGZqppFwLwcBC1JQ6Vr   
1                             Smells Like Teen Spirit  75aLTVBSGIquqzQ6AkmK3Q   
2                                            All Star  3cfOd4CMv2snFaKAnMdnvK   
3                                    Under the Bridge  3d9DChrdc6BOeFsbrZ3Is0   
4                                                Iris  6Qyc6fS4DsZjB2mRW9DsQs   
5                                               Creep  6b2oQwSGFkzsMtQruIWm2p   
6                                  Losing My Religion  6FBlNlVx0xrXPb9suicaMM   
7                                Nothing Else Matters  0nLiqZ6A27jJri2VCalIUs   
8                                     Come As You Are  7aCSKL5LmVhakVs0l483B5   
9                        I Don't Want To Miss A Thing  1hyuZzPDnSpBBPVztdGDWr   
10            Bitter Sweet Symphony - Remastered 2016  57iDDD9N9tTWe75x6qhStw   
11                          

In [ ]:
df.to_csv('playlist_songs4.csv', index=False)

In [ ]:
def get_all_user_tracks():
    data = []

    playlists = sp.current_user_playlists()
    for playlist in playlists['items']:
        playlist_id = playlist['id']
        playlist_name = playlist['name']

        results = sp.playlist_tracks(playlist_id)
        tracks = results['items']

        while results['next']:
            results = sp.next(results)
            tracks.extend(results['items'])

        for item in tracks:
            track = item['track']
            if track:  
                song_name = track['name']
                song_id = track['id']
                duration = track['duration_ms'] // 1000  # Convert ms to seconds
                artists = [artist['name'] for artist in track['artists']]
                album = track['album']['name']
                release_date = track['album']['release_date']

                artist_id = track['artists'][0]['id']
                artist_info = sp.artist(artist_id)
                genres = artist_info['genres']

                
                data.append({
                    'Song Name': song_name,
                    'Song ID': song_id,
                    'Artists': artists,
                    'Genres': genres,
                    'Time (seconds)': duration,
                    'Release Date': release_date,
                    'Album': album,
                    'Playlist ID': playlist_id,
                    'Playlist Name': playlist_name
                })

    saved_albums = sp.current_user_saved_albums()
    for album_item in saved_albums['items']:
        album = album_item['album']
        album_name = album['name']
        album_tracks = album['tracks']['items']

        for track in album_tracks:
            song_name = track['name']
            song_id = track['id']
            duration = track['duration_ms'] // 1000
            artists = [artist['name'] for artist in track['artists']]
            release_date = album['release_date']

            artist_id = track['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres = artist_info['genres']

            # Append the data
            data.append({
                'Song Name': song_name,
                'Song ID': song_id,
                'Artists': artists,
                'Genres': genres,
                'Time (seconds)': duration,
                'Release Date': release_date,
                'Album': album_name,
                'Playlist ID': None,
                'Playlist Name': None
            })

    df = pd.DataFrame(data)
    df.to_csv('user_library_tracks.csv', index=False)
    print("Data saved to user_library_tracks.csv")
    return df

df = get_all_user_tracks()
print(df.head())